In [1]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'Intel-Image-Clf'

In [2]:
def load_data():
    X = []
    y = []
    labels = {}
    idx = -1
    for folder in tqdm(os.listdir('./data/')):
        idx += 1
        labels[folder] = idx
        for file in os.listdir(f'./data/{folder}/'):
            file = f'./data/{folder}/{file}'
            img = cv2.imread(file)
            img = cv2.resize(img,(224,224))
            X.append(img/255.0)
            y.append(labels[folder])
    return X,y,idx,labels

In [ ]:
X,y,idx,labels = load_data()

 33%|███▎      | 2/6 [00:03<00:07,  1.97s/it]

In [ ]:
idx

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=False)

In [ ]:
def get_loss(preds,y,criterion):
    loss = criterion(preds,y)
    return loss.item()
def get_accuracy(preds,y):
    total = -1
    correct = -1
    for pred,y_batch in zip(preds,y):
        pred = round(float(pred))
        y_batch = round(float(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    return round(total/correct,3)

In [ ]:
def pred_test_data(model):
    preds = []
    for file in os.listdir('./test_data/'):
        file = f'./test_data/{file}'    
        img = cv2.imread(file)
        img = cv2.resize(img,(224,224))
        pred = model(img)
        plt.figure(figsize=(12,7))
        plt.showfig(img)
        plt.title(f'{labels[torch.argmax(pred)]}')
        plt.savefig(f'./preds/{file}')
        plt.close()
        preds.append(torch.argmax(pred))
    return preds

In [ ]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = MaxPool2d((2,2),(2,2))
        self.activation = ReLU()
        self.conv1 = Conv2d(3,8,(3,3))
        self.conv1batchnorm = BatchNorm2d(8)
        self.conv2 = Conv2d(8,12,(3,3))
        self.conv2batchnorm = BatchNorm2d(12)
        self.conv3 = Conv2d(12,16,(5,5))
        self.conv3batchnorm = BatchNorm2d(16)
        self.conv4 = Conv2d(16,20,(5,5))
        self.conv4batchnorm = BatchNorm2d(20)
        self.linear1 = Linear(20*5*3,32)
        self.linear1batchnorm = BatchNorm1d(32)
        self.linear2 = Linear(32,64)
        self.linear2batchnorm = BatchNorm1d(64)
        self.linear3 = Linear(64,128)
        self.linear3batchnorm = BatchNorm1d(128)
        self.linear4 = Linear(128,256)
        self.linear4batchnorm = BatchNorm1d(256)
        self.linear5 = Linear(256,128)
        self.linear5batchnorm = BatchNorm1d(128)
        self.output = Linear(128,idx)
        
    def forward(self,X):
        preds = X
        preds = self.max_pool2d(self.activation(self.conv1batchnorm(self.conv1(preds))))
        preds = self.max_pool2d(self.activation(self.conv2batchnorm(self.conv2(preds))))
        preds = self.max_pool2d(self.activation(self.conv3batchnorm(self.conv3(preds))))
        preds = self.max_pool2d(self.activation(self.conv4batchnorm(self.conv4(preds))))
        print(preds.shape)
        preds = preds.view(-1,20*5*3)
        preds = self.activation(self.linear1batchnorm(self.linear1(preds)))
        preds = self.activation(self.linear2batchnorm(self.linear2(preds)))
        preds = self.activation(self.linear3batchnorm(self.linear3(preds)))
        preds = self.activation(self.linear4batchnorm(self.linear4(preds)))
        preds = self.activation(self.linear5batchnorm(self.linear5(preds)))
        preds = self.output(preds)
        return preds

In [ ]:
model = Model()

In [ ]:
model.to(device)

In [ ]:
criterion = CrossEntropyLoss()

In [ ]:
from torch.optim import *

In [ ]:
optimizer = Adam(model.parameters(),lr=0.001)

In [ ]:
epochs = 100

In [ ]:
batch_size=  32

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+bacth_size].to(device).float().view(-1,3,224,224)
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        preds = preds.to(device)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'Loss':loss.item()})
        wandb.log({'Val Loss':get_loss(model(X_test),y_test,criterion)})
        wandb.log({'Accuracy':get_accuracy(preds,y_batch)})
        wandb.log({'Val Accuracy':get_accuracy(model(X_test),y_test)})
        pred_test_data(model)
        for file in os.listdir('./preds/'):
            wandb.log({f'/Img/{file}':wandb.Image(cv2.imread(f'./preds/{file}'))})
wandb.watch(model)
wandb.finish()